In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
from jupyterthemes import jtplot

from smartenergy.database import Stations, HourlyMeasurements
jtplot.style('onedork')

np.warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
display(HTML("<style>.container { width: 1480px !important; }</style>"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
station_ids = HourlyMeasurements().station_ids
stations = Stations()
station_ids

['190',
 '30',
 '309',
 '32',
 '34',
 '344',
 '345',
 '366',
 '369',
 '37',
 '40',
 '43',
 '47',
 '48',
 '68',
 '71',
 '97']

In [8]:
from math import ceil
width = 3
length = ceil(len(station_ids)/width)
grid = np.zeros((length, width))
for i in range(length):
    grid_elements = station_ids[(i*width):((i+1)*width)]
    grid[i, :len(grid_elements)] = grid_elements

grid_dict = {}
for row in range(len(grid)):
    for col in range(len(grid[row])):
        if grid[row, col] == 0:
            continue
            
        left_node = (row, col - 1)
        upper_node = (row - 1, col)
        right_node = (row, col + 1)
        lower_node = (row + 1, col)
        grid_dict[str(int(grid[row, col]))] = {str(int(grid[_row, _col])): 0.9 for _row, _col in [left_node, upper_node, right_node, lower_node]
                                               if 0 <= _row <= (length-1) and 0 <= _col <= (width-1) and grid[_row, _col] != 0}
        
print(grid_dict)

{'190': {'30': 0.9, '32': 0.9}, '30': {'190': 0.9, '309': 0.9, '34': 0.9}, '309': {'30': 0.9, '344': 0.9}, '32': {'190': 0.9, '34': 0.9, '345': 0.9}, '34': {'32': 0.9, '30': 0.9, '344': 0.9, '366': 0.9}, '344': {'34': 0.9, '309': 0.9, '369': 0.9}, '345': {'32': 0.9, '366': 0.9, '37': 0.9}, '366': {'345': 0.9, '34': 0.9, '369': 0.9, '40': 0.9}, '369': {'366': 0.9, '344': 0.9, '43': 0.9}, '37': {'345': 0.9, '40': 0.9, '47': 0.9}, '40': {'37': 0.9, '366': 0.9, '43': 0.9, '48': 0.9}, '43': {'40': 0.9, '369': 0.9, '68': 0.9}, '47': {'37': 0.9, '48': 0.9, '71': 0.9}, '48': {'47': 0.9, '40': 0.9, '68': 0.9, '97': 0.9}, '68': {'48': 0.9, '43': 0.9}, '71': {'47': 0.9, '97': 0.9}, '97': {'71': 0.9, '48': 0.9}}


In [12]:
stations_dict = list()
for i, station in enumerate(station_ids):
    stations_dict.append({
        'solbox_id': station,
        'res_gen_bat': 0.95,
        'res_gen_con': 0.95,
        'res_gen_grid': 0.95,
        'res_bat_con': 0.95,
        'res_bat_grid': 0.95,
    })
    for s, res in grid_dict[station].items():
        stations_dict[i]['res_self_{}'.format(s)] = res

stations_dict

[{'solbox_id': '190',
  'res_gen_bat': 0.95,
  'res_gen_con': 0.95,
  'res_gen_grid': 0.95,
  'res_bat_con': 0.95,
  'res_bat_grid': 0.95,
  'res_self_30': 0.9,
  'res_self_32': 0.9},
 {'solbox_id': '30',
  'res_gen_bat': 0.95,
  'res_gen_con': 0.95,
  'res_gen_grid': 0.95,
  'res_bat_con': 0.95,
  'res_bat_grid': 0.95,
  'res_self_190': 0.9,
  'res_self_309': 0.9,
  'res_self_34': 0.9},
 {'solbox_id': '309',
  'res_gen_bat': 0.95,
  'res_gen_con': 0.95,
  'res_gen_grid': 0.95,
  'res_bat_con': 0.95,
  'res_bat_grid': 0.95,
  'res_self_30': 0.9,
  'res_self_344': 0.9},
 {'solbox_id': '32',
  'res_gen_bat': 0.95,
  'res_gen_con': 0.95,
  'res_gen_grid': 0.95,
  'res_bat_con': 0.95,
  'res_bat_grid': 0.95,
  'res_self_190': 0.9,
  'res_self_34': 0.9,
  'res_self_345': 0.9},
 {'solbox_id': '34',
  'res_gen_bat': 0.95,
  'res_gen_con': 0.95,
  'res_gen_grid': 0.95,
  'res_bat_con': 0.95,
  'res_bat_grid': 0.95,
  'res_self_32': 0.9,
  'res_self_30': 0.9,
  'res_self_344': 0.9,
  'res_self_

In [13]:
stations.insert_many(stations_dict)